This notebook provides a step-by-step guide on how to use ai4stem for analyzing exerimental images.

For a quick start, the following code may be uncommented and executed:

# Quick start

After specifying an input image (here, Fe bcc [100]), the following code can be used to analyze it via ai4stem, employing a pretrained model (the one employed in the corresponding publication):

In [ ]:
from ai4stem.utils.utils_data import load_pretrained_model, load_example_image
from ai4stem.utils.utils_spm import predict

input_image = load_example_image()
model = load_pretrained_model()

sliced_images, fft_descriptors, prediction, uncertainty = predict(input_image, model)

Note that the model is trained on a specific pixel/angstrom relation. Specifically, the model is trained to classify local windows of size 12 Angstrom, which corresponds to 100 pixels in the simulation settings that we employed for creating the training set. If a different resolution is employed, we recommend to adapt the window size by , (i.e., pixel/angstrom relation) is employed for the user's application, we recommend to adapt the window size to a We recommend to adapt the window size by  or escale the images

More detailed explanations are provided in the following.

# Step-by-step explanations

First we import some packages:

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import cv2
from collections import defaultdict
from copy import deepcopy

from ai4stem.utils.utils_data import load_pretrained_model, load_example_image
from ai4stem.utils.utils_fft import calc_fft
from ai4stem.utils.utils_spm import localwindow
from ai4stem.utils.utils_nn import predict_with_uncertainty

# Necessary specifications

In [ ]:
# Specify path where to save the results:
results_folder = '.'

input_image = load_example_image()
image_name = 'Fe_bcc'
pixel_to_angstrom = 0.12452489444788318
window_size = 12.
stride_size = [36, 36]

# If want to visualize local windows, set to true
save_local_windows = False
local_windows_path = '.'

In [ ]:
plt.imshow(input_image, cmap='gray')
plt.show()

The following cells do not have to be changed:

In [ ]:
filenames = {image_name: (input_image, 
                               pixel_to_angstrom)}


In [ ]:
# load pretrained model
model = load_pretrained_model()
model_name = 'pretrained_model'
model.summary()

# Analyze image

In [ ]:
# FFT HAADF descriptor settings
sigma = None # optional parameter
thresholding = True # very important
n_iter = 100 # MC dropout samples


counter = 0
results_dict = defaultdict(dict)
for key in filenames:

    dx_origin = filenames[key][1]
    filename = filenames[key][0]
    name = key
    
    if type(filename) == str:
        
        img = cv2.imread(filename)
        image = deepcopy(img[:, :, 0])
    else:
        image = filename
    adapted_window_size = window_size * (1. / dx_origin)
    adapted_window_size = int(round(adapted_window_size))
    print('For image called {}, window {} [Angstrom] corresponds to {} pixels'.format(key, window_size, adapted_window_size))

    sliced_images, spm_pos, ni, nj = localwindow(image, stride_size=stride_size, pixel_max=adapted_window_size)
    np.save(os.path.join(results_folder, '{}_{}_images.npy'.format(name,
                                                                       model_name)), sliced_images)
    
    fft_descriptors = []
    for im in sliced_images:
        fft_desc = calc_fft(im, sigma=sigma, thresholding=thresholding)
        fft_descriptors.append(fft_desc)
    np.save(os.path.join(results_folder, '{}_fft_desc.npy'.format(name)), np.asarray(fft_descriptors))

    repeated_images = np.array([np.stack([_]) for _ in fft_descriptors])
    repeated_images = np.moveaxis(repeated_images, 1, -1)


    prediction, uncertainty = predict_with_uncertainty(repeated_images, 
                                                   model=model, 
                                                   model_type='classification', 
                                                   n_iter=n_iter)
    np.save(os.path.join(results_folder, '{}_{}_predictions.npy'.format(name,
                                                                       model_name)), prediction)
    for key in uncertainty:
        np.save(os.path.join(results_folder, '{}_{}_{}.npy'.format(name,
                                                                  model_name,
                                                                  key)), uncertainty[key])
    argmax_pred = prediction.argmax(axis=-1)
    argmax_pred = np.reshape(argmax_pred, (ni, nj))
    mutinfo = uncertainty['mutual_information']
    mutinfo = np.reshape(mutinfo, (ni, nj))

    results_dict[model_name][name] = {}
    results_dict[model_name][name]['Prediction'] = argmax_pred
    results_dict[model_name][name]['Mutual information'] = mutinfo
    results_dict[model_name][name]['Input Image'] = image

# Visualize predictions

In [ ]:
import matplotlib

matplotlib.rcParams.update({'font.size': 10})


for key in filenames:
    name = key

    argmax_pred = results_dict[model_name][name]['Prediction']
    mutinfo = results_dict[model_name][name]['Mutual information']
    image = results_dict[model_name][name]['Input Image']

    fig, axs = plt.subplots(1, 3, figsize=(10, 10))


    im1 = axs[0].imshow(image, cmap='gray')
    fig.colorbar(im1, ax=axs[0], orientation='vertical')

    im2 = axs[1].imshow(argmax_pred, cmap='tab20')
    fig.colorbar(im2, ax=axs[1],  orientation='vertical')

    im3 = axs[2].imshow(mutinfo, cmap='hot', vmin=0.0)
    fig.colorbar(im3, ax=axs[2],  orientation='vertical')
    
    axs[0].axis('off')
    axs[1].axis('off')
    axs[2].axis('off')

    fig.tight_layout()
    
    plt.show()

